# ============================ WHY I'M KEEPING THIS ====================================
# You've already gotten rid of using this method below, but you might need a part of it 
# when you implement the next step in your methodology (where you break into durations
# based on storms)
# ======================================================================================

In [7]:
# import numpy as np
# import pandas as pd

# # filename should be type string ex. 'precipitation.csv' and file path should be in relation to where this file is stored
# def TStoStorms(filename, dryTimeBetweenStorms):
#     '''accepts a precipitation time series and number of dry time steps before a storm ends and splits it into individual storms'''
    
#     # read data from csv
#     headers = ['datetime', 'pr']
#     dtypes = {'col1': 'str', 'col2': 'float'}
#     date_cols = ['datetime']
#     storms_data = pd.read_csv(filename, dtype = dtypes, header=None, names = headers, parse_dates = date_cols)

#     storms_data.loc[ : , 'pr'] = storms_data.loc[ : , 'pr'] / 25.4


#     dry = dryTimeBetweenStorms # counter to iterate only on dry days

#     # establish big list (storms) which will contain all storms, separated
#     # establish little list (current_storm) which will hold precipitation for one storm until it goes to big list
#     storms = []
#     current_storm = []
#     storms.append(current_storm)

#     for value in storms_data.loc[ : , 'pr']:
#         dry += 1 # count the number of consecutive dry steps


#         # if precipitation is higher than a certain threshold, reset dry counter
#         if value >= 0.01: 
#             dry = 0

#         # when during a storm, append the current precip value to small list, which is continuously updating big list simultaneously
#         if dry < dryTimeBetweenStorms: 
#             current_storm.append(value)

#         # when the dry time reaches the threshold to end a storm, begin a new small list that is being appended to big list
#         elif dry == dryTimeBetweenStorms: 
#             current_storm = []
#             storms.append(current_storm)

#     #     print(dry, value) # by using this, you can see that it does take need dryTimeBetweenStorms time steps to all be zero before moving to the next storm, it just doesn't count them all in that storm's list entry

#     return(storms)
    
    
# TStoStorms('sample_precip1.csv', 5);


In [8]:
# ============================ WHY I'M KEEPING THIS ====================================
# Within the loop at the end, there's some helpful pandas code structure for mutating
# dataframes in place.
# ======================================================================================


# import math
# import pandas as pd
# import numpy as np
# def TStoIDF(past_filename, future_filename):
#     '''This reads in data from a csv, assigns variable types, calculates CDF shape factors, then solves for past and future precip values.
#        Their ratio is the delta factor'''
    
#     # read data from CSVs
#     headers = ['datetime', 'pr']
#     dtypes = {'col1': 'str', 'col2': 'float'}
#     date_cols = ['datetime']
#     past_storms_data = pd.read_csv(past_filename, dtype = dtypes, header=None, names = headers, parse_dates = date_cols)
#     future_storms_data = pd.read_csv(future_filename, dtype = dtypes, header=None, names = headers, parse_dates = date_cols)
    
#     # convert units
# #     storms_data.loc[ : , 'pr'] = storms_data.loc[ : , 'pr'] / 25.4
    
#     # calculate statistics based on full precip inputs
#     past_mean_pr = np.mean(past_storms_data.loc[ : , 'pr'])
#     past_std_pr = np.std(past_storms_data.loc[ : , 'pr'])
    
#     future_mean_pr = np.mean(future_storms_data.loc[ : , 'pr'])
#     future_std_pr = np.std(future_storms_data.loc[ : , 'pr'])    
    
#     # calculate shape factors from full input
#     ##### these are the equations used
#     ### original
#     #     mean = mu + beta*gamma
#     #     std = pi*beta/sqrt(6)   
#     ### solved for shape factors 
#     #     mu = mean - (beta*gamma)
#     #     beta = std*sqrt(6)/pi 
#     euler_mascheroni_constant = 0.57721
    
#     past_beta = past_std_pr * math.sqrt(6) / math.pi
#     past_mu = past_mean_pr - past_beta * euler_mascheroni_constant

#     future_beta = future_std_pr * math.sqrt(6) / math.pi
#     future_mu = future_mean_pr - future_beta * euler_mascheroni_constant
    
  
    
#     # calculate CDF value for each row (unordered)
#     projected_storms_data = past_storms_data
#     for count, value in enumerate(projected_storms_data.loc[ : ,'pr']):
#         projected_storms_data.loc[count, 'CDF'] = math.exp(-1 * math.exp(-1 * (value - past_mu) / past_beta))
#         projected_storms_data.loc[count, 'Projected pr'] = -1 * future_beta * (math.log(-1 * math.log(projected_storms_data.loc[count, 'CDF']))) + future_mu
#         projected_storms_data.loc[count, 'Delta'] = projected_storms_data.loc[count, 'Projected pr'] / projected_storms_data.loc[count, 'pr']
#     print(projected_storms_data)
    
#     # to do: change this to run not on every line of precipitation, but instead run for each storm? I think?
#     # so it'd be like the total depth of a 24-hr storm was 2 inches in the past, but then it gets projected into the future and would have
#     # an equivalent depth of 2.5 inches, so the delta factor would be 2.5 / 2.0 = 1.25 for this one 24-hr storm. Would also need to find the 
#     # return period for this storm based on the historical 24-hr depth. This way, a regression can be used to find a line for each delta
#     # vs. return period for this duration
    
    
# TStoIDF('sample_past_precip.csv', 'sample_future_precip.csv')

Goal:
- read in model data
- parse the data, separating it into storms
- bin storms by their durations
- find deltas based on binned storms

In [9]:
def read_csv(filename):
    '''Read in data from csv, assign headers, set datatypes, perform unit conversions if necessary, and find time step for
       the dataset in minutes.'''
# read data from csv
    headers = ['datetime', 'pr']
    dtypes = {'col1': 'str', 'col2': 'float'}
    date_cols = ['datetime']
    storms_data = pd.read_csv(filename, dtype = dtypes, header=None, names = headers, parse_dates = date_cols)
    
    # perform unit conversions
    storms_data.loc[ : , 'pr'] = storms_data.loc[ : , 'pr']

    # determine time step by subtracting one datetime from another
    TS = storms_data.loc[1, 'datetime'] - storms_data.loc[0, 'datetime']
    # convert time delta type to int type in minutes
    TS = int(TS.total_seconds() / 60)
    
    return storms_data

storms_data = read_csv('sample_past_precip.csv')
# storms_data

FileNotFoundError: [Errno 2] No such file or directory: 'sample_past_precip.csv'

In [10]:
import xarray as xr
import numpy as np
import pandas as pd
import packaging
import matplotlib.pyplot as plt
import netCDF4
import math

def read_netcdf4(filename):
    '''Read in data from netcdf4, assign headers, set datatypes, perform unit conversions if necessary'''
    # read in netcdf4 data to xarray
    storms_data = xr.open_dataset(filename)
    # convert xarray dataarray to pandas dataframe
    storms_data = storms_data.RAINRATE.to_dataframe()
    # get rid of lat long columns that could stay without causing any issues, but I spent ten minutes trying to get it to work, 
    # so they will remain removed
    storms_data = storms_data.drop(['lat', 'lon'], axis=1)
    # default index is multiindex. drop the additonal levels
    storms_data = storms_data.droplevel(['x', 'y'])
    # datetime was being used as the index, not one of the columns. Change it into a column and reset index
    storms_data['datetime'] = storms_data.index
    storms_data = storms_data.reset_index(drop = True)
    # convert datetime column from string to datetime
    storms_data['datetime']= pd.to_datetime(storms_data['datetime'])
    # rename columns
    storms_data = storms_data.rename(columns={"RAINRATE": "pr"})
    # reorder columns
    storms_data = storms_data.iloc[:,[1, 0]]

#     # perform unit conversions if necessary
#     # default units for netcdf4 are mm to 0.1 mm accuracy
#     storms_data.loc[ : , 'pr'] = storms_data.loc[ : , 'pr']

    return storms_data

storms_data = read_netcdf4('CONUS404-19791001-20221001-Dover-Air-Force-Base.nc')

display(storms_data)

This cell is used only while I don't have the full future CONUS404 dataset. It splits
the past data in half and pretends that the latter part is from the future.

In [11]:
# splitting dataframe by row index
past_storms_data = storms_data.iloc[:188471,:]
future_storms_data = storms_data.iloc[188471:,:]
future_storms_data.reset_index(inplace = True, drop = True)
# print(past_storms_data = df.iloc[:188471,:]

print("Shape of new dataframes - {} , {}".format(past_storms_data.shape, future_storms_data.shape))

Finds the timestep (in minutes) of the given data by subtracting two datetimes

In [12]:
def find_timestep(storms_data):
    '''Find time step of dataset in minutes'''
    # determine time step by subtracting one datetime from another
    TS = storms_data.iloc[1, 0] - storms_data.iloc[0, 0]
    # convert timedelta type to int type in minutes
    TS = TS.total_seconds() / 60
    return TS

past_TS = find_timestep(past_storms_data)
past_TS

60.0

Test out using different values in the storm durations list and see how that impacts the results.
ex: most of the storms were 6hrs in length, so maybe try three durations, <6 hrs, 6 hrs, and >6 hrs

Note: .copy() creates a copy of something that is not linked to the original

In [ ]:
# Create set that contains all the storm durations being used in minutes
stormDurations = [60, 120, 180, 360, 720, 1440] # durations every storm will be grouped into (minutes)

def separate_into_durations(storms_data,stormDurations, TS):
    '''Read in precip data. Put it into rolling sums for each storm and grab the highest rainfall sum for each storm duration
       so you can use these max values to calculate the CDF shape parameters.'''
   
    rolling_avgs_all_durations = []
    full_rolling = pd.DataFrame()
    for duration in stormDurations:
        timesteps = int(duration / TS)

        # initialize empty list
        rolling_list = []
        # loop through precip tables to compute rolling sums and grab datetimes
        for count, value in enumerate(storms_data.loc[ : ,'pr']):
            # will grab the start time for each rolling sum and store in a list
            rolling_start_datetime = storms_data.loc[: , 'datetime']
            # will do a rolling sum through a series
            rolling_sum = storms_data.iloc[count:timesteps + count, 1].sum(axis=0)
            # will compile a list of the rolling sums
            rolling_list.append(rolling_sum)     

        # create df from the list containing the rolling sums
        df_list = pd.DataFrame(rolling_list)
        # add column with current duration
        df_list['duration'] = duration
        
        # combine the two dataframes (starting datetime and rolling sum) to get desired output for this duration
        rolling_avgs = pd.concat([rolling_start_datetime, df_list], axis=1)
        
        # store this df so it's not overwritten next loop. This is a bad way of doing this computationally, so fix if 
        # optimization is needed
        full_rolling = pd.concat([full_rolling, rolling_avgs], ignore_index = True)
        
    # rename columns
    full_rolling = full_rolling.rename(columns = {'datetime': 'starting datetime', 0: 'pr_sum'})
    
    # find max value of pr_sum for each year. This doesn't specify which year it is, but that shouldn't matter
    max = full_rolling.groupby(['duration', lambda x: full_rolling['starting datetime'][x].year], as_index = False)["pr_sum"].max()
    return max

past_max_each_year = separate_into_durations(past_storms_data, stormDurations, past_TS)
display(past_max_each_year)

In [ ]:
# run these two functions

future_TS = find_timestep(future_storms_data)
future_max_each_year = separate_into_durations(future_storms_data, stormDurations, future_TS)

In [ ]:
copy_past_max_each_year = pd.DataFrame(columns = ['duration', 'pr_sum'], index = list(range(len(past_max_each_year.index))))
for count in past_max_each_year.index:
    copy_past_max_each_year['duration'].iloc[count] = past_max_each_year['duration'].iloc[count]
    copy_past_max_each_year['pr_sum'].iloc[count] = past_max_each_year['pr_sum'].iloc[count]
copy_past_max_each_year = copy_past_max_each_year.apply(pd.to_numeric)
copy_past_max_each_year

copy_future_max_each_year = pd.DataFrame(columns = ['duration', 'pr_sum'], index = list(range(len(future_max_each_year.index))))
for count in future_max_each_year.index:
    copy_future_max_each_year['duration'].iloc[count] = future_max_each_year['duration'].iloc[count]
    copy_future_max_each_year['pr_sum'].iloc[count] = future_max_each_year['pr_sum'].iloc[count]
copy_future_max_each_year = copy_future_max_each_year.apply(pd.to_numeric)
copy_future_max_each_year
# display(copy_future_max_each_year)

def compute_CDF(past_max_each_year, future_max_each_year):
    '''Calculate CDF shape parameters and deltas for each duration'''
#     display(future_max_each_year)
#     display(past_max_each_year)

    # calculate statistics for each storm duration based on full rolling input of ~past~ data
    past_mean_pr = past_max_each_year.groupby(['duration']).mean()
    past_std_pr  = past_max_each_year.groupby(['duration']).std()
    euler_mascheroni_constant = 0.57721566490153286060651209008240243104215933593992

    # calculate shape factors from statistics for each duration
    past_beta = past_std_pr * math.sqrt(6) / math.pi
    past_mu = past_mean_pr - past_beta * euler_mascheroni_constant
    
    # calculate statistics for each storm duration based on full rolling input of ~future~ data
    future_mean_pr = future_max_each_year.groupby(['duration']).mean()
    future_std_pr  = future_max_each_year.groupby(['duration']).std()
    euler_mascheroni_constant = 0.57721566490153286060651209008240243104215933593992
#     print(future_mean_pr, future_std_pr)

    # calculate shape factors from statistics for each duration
    future_beta = future_std_pr * math.sqrt(6) / math.pi
    future_mu = future_mean_pr - future_beta * euler_mascheroni_constant
#     print(future_beta, future_mu)
#     print("This is beta", beta)
#     print(mu)
#     print(beta.iloc[1])

    # these are the equations used
#     mean = mu + beta*gamma
#     std = pi*beta/sqrt(6) 
#
#     Rearranged:
#     mu = mean - (beta*gamma)
#     beta = std*sqrt(6)/pi  

# ***************************************** here is where you find the deltas ********************************************

#     past_max_each_year['CDF'] = 0
# #     past_max_each_year.drop(['past_CDF'], axis = 1)
# #     print(past_max_each_year)
#     for count, value in enumerate(past_max_each_year['pr_sum']):
# #         print(value)
# # #         print(past_max_each_year['duration'].iloc[count])
#         past_max_each_year['CDF'].iloc[count] = math.exp(-1 * math.exp(-1 * (value - mu.loc[past_max_each_year['duration'].iloc[count]]) / beta.loc[past_max_each_year['duration'].iloc[count]]))
# #         print(beta.loc[past_max_each_year['duration'].iloc[count]])
# #         print(mu.loc[past_max_each_year['duration'].iloc[count]])
            
#     print(beta)
    # calculate CDF value for each row (unordered)
#     projected_storms_data = past_storms_data
    for count, value in enumerate(past_max_each_year['pr_sum']):
#         print(beta.loc[past_max_each_year['duration'].iloc[count]])
        past_max_each_year.loc[count, 'CDF'] = math.exp(-1 * math.exp(-1 * (value - past_mu.loc[past_max_each_year['duration'].iloc[count]]) / past_beta.loc[past_max_each_year['duration'].iloc[count]]))
        past_max_each_year.loc[count, 'projected_pr'] = float(-1 * future_beta.loc[past_max_each_year['duration'].iloc[count]] * (math.log(-1 * math.log(past_max_each_year.loc[count, 'CDF']))) + future_mu.loc[past_max_each_year['duration'].iloc[count]])
#         print(past_max_each_year.loc[count, 'CDF'])
#         past_max_each_year.loc[count, 'projected_pr'] = future_beta.loc[past_max_each_year['duration'].iloc[count]]
#         print(-1 * future_beta.loc[past_max_each_year['duration'].iloc[count]] * (math.log(-1 * math.log(past_max_each_year.loc[count, 'CDF']))) + future_mu.loc[past_max_each_year['duration'].iloc[count]])
#         print(future_beta.loc[past_max_each_year['duration'].iloc[count]], future_mu.loc[past_max_each_year['duration'].iloc[count]], past_max_each_year.loc[count, 'CDF'])
    
        past_max_each_year.loc[count, 'delta'] = past_max_each_year.loc[count, 'projected_pr'] / past_max_each_year.loc[count, 'pr_sum']
#     print(past_max_each_year)

#     print(beta.loc[60])
#     print(beta)
#     past_max_each_year
    
    return past_max_each_year

deltas = compute_CDF(copy_past_max_each_year, copy_future_max_each_year) #!!!!!!!!!!!!!!!!!!!!!!!!!!change this!!!!!!!!!!!!!!!!!!!!!!!!
display(deltas)

In [ ]:
import pandas as pd
def read_historical_csv(filename):
    '''Read observed historical precipitation time series from .csv'''
    
    # read data from csv
    headers = ['datetime', 'pr']
    dtypes = {'col1': 'str', 'col2': 'float'}
    date_cols = ['datetime']
    historical = pd.read_csv(filename, dtype = dtypes, header=None, names = headers, parse_dates = date_cols)
    
    # convert units from mm to mm/hr by dividing by 5-min TS
    historical.pr = historical.pr * (60 / hist_TS)

    return historical
historical = 0
historical = read_historical_csv('2012_historical.csv')
# historical = read_historical_csv('sample_past_precip.csv')
# print(historical)

hist_TS = find_timestep(historical)
print(hist_TS)

In [3]:
dry_time_between_storms = 360 # number of minutes of dry time before the end of a storm

def discretize_by_storm(historical, dry_time_between_storms, TS):
    '''Read in historical precipitation TS and assign numbers to each storm that occurs in a new column.'''
    
    storm_no = 1 # set starting point for labelling the number of storms in your time series
    dry_counter = 0 # intialize counter that counts number of consecutive dry time steps
    historical['storm_no'] = "There is nothing here." # create new empty column
    
    for count, precip in enumerate(historical['pr']):
        dry_counter += 1
        if precip > 0:
            dry_counter = 0

        if dry_counter == dry_time_between_storms / hist_TS:
            storm_no += 1
        historical['storm_no'].iloc[count] = storm_no
#         print(precip, storm_no)
        
    return historical
discretized_historical = 0
# discretized_historical = discretize_by_storm(historical, dry_time_between_storms, hist_TS)
# display(discretized_historical)

Discretize model storms !! experimental work !!

In [ ]:
discretized_modeled_past = 0
discretized_modeled_future = 0
dry_time_between_storms = 360
modeled_TS = find_timestep(past_storms_data)
discretized_modeled_past = discretize_by_storm(past_storms_data)
discretized_modeled_future = discretize_by_storm(future_storms_data)

trimmed_discretized_modeled_past = trim_to_storms(discretized_modeled_past, dry_time_between_storms, modeled_TS)
trimmed_discretized_modeled_future = trim_to_storms(discretized_modeled_future, dry_time_between_storms, modeled_TS)

rounded_trimmed_discretized_modeled_past = round_to_durations(trimmed_discretized_modeled_past, stormDurations)
rounded_trimmed_discretized_modeled_future = round_to_durations(trimmed_discretized_modeled_future, stormDurations)






In [4]:
def trim_to_storms(discretized_historical, dry_time_between_storms, TS):
    '''Reads in discretized storms and trims out long periods of dry time between them. Returns a table with the duration
       of each storm in minutes rounded to the nearest usable length.'''
    dry_counter = 0 # intialize counter that counts number of consecutive dry time steps
    
    for count, precip in enumerate(discretized_historical['pr']):
        dry_counter += 1
        if precip > 0:
            dry_counter = 0

    #     print(precip, dry_counter)
        if dry_counter >= dry_time_between_storms / hist_TS:
            discretized_historical = discretized_historical.drop([count])
#         print(precip, dry_counter)


    storm_durations = discretized_historical.groupby(['storm_no']).count() * TS
    storm_durations['duration'] = storm_durations['datetime']
    storm_durations = storm_durations.drop(['datetime', 'pr'], axis = 1)
    
#     display(storm_durations)
    
    storm_sums = discretized_historical.groupby(['storm_no'])['pr'].sum()
    storm_sums = storm_sums.to_frame()
    storm_sums['pr_sum'] = storm_sums['pr']
    storm_sums = storm_sums.drop(['pr'], axis = 1)

#     display(storm_sums)
    
    merge_durations_sums = storm_sums.merge(storm_durations, left_on = 'storm_no', right_on = 'storm_no')
#     display(merge_durations_sums)

#     storm_durations['duration'] = storm_durations['datetime']
#     storm_durations = storm_durations.drop(['pr', 'datetime'], axis = 1)
    
#     display(storm_durations, storm_sums)

    return merge_durations_sums

# trimmed_discretized_historical = trim_to_storms(discretized_historical, dry_time_between_storms, hist_TS)
# trimmed_discretized_historical

In [5]:
def round_to_durations(trimmed_discretized_historical, durations):
    '''Rounds the duration of every storm to its closest accepted value.'''
    rounded_trimmed_discretized_historical = trimmed_discretized_historical
    for count, value in enumerate(rounded_trimmed_discretized_historical['duration']):
        if 60 <= value < 90:
            value = 60
            
        elif 90 <= value < 150:
            value = 120 
            
        elif 150 <= value < 270:
            value = 180
            
        elif 270 <= value < 540:
            value = 360
            
        elif 540 <= value < 1080:
            value = 720
            
        else:
            value = 1440
            
        rounded_trimmed_discretized_historical['duration'].iloc[count] = value
        
    return rounded_trimmed_discretized_historical

# rounded_trimmed_discretized_historical = round_to_durations(trimmed_discretized_historical, stormDurations)
# rounded_trimmed_discretized_historical

In [ ]:
def merge_dfs(discretized_historical, rounded_trimmed_discretized_historical):
    merged = discretized_historical.merge(rounded_trimmed_discretized_historical, left_on = 'storm_no', right_on = 'storm_no')

    return merged
merged_historical = merge_dfs(discretized_historical, rounded_trimmed_discretized_historical,)
merged_historical

,datetime,pr,storm_no,pr_sum,duration
0,2012-01-01 00:00:00,0.0,1,0.00,360.0
1,2012-01-01 00:05:00,0.0,1,0.00,360.0
2,2012-01-01 00:10:00,0.0,1,0.00,360.0
3,2012-01-01 00:15:00,0.0,1,0.00,360.0
4,2012-01-01 00:20:00,0.0,1,0.00,360.0
...,...,...,...,...,...
103763,2012-12-27 10:25:00,0.0,115,555.96,1440.0
103764,2012-12-27 10:30:00,0.0,115,555.96,1440.0
103765,2012-12-27 10:35:00,0.0,115,555.96,1440.0
103766,2012-12-27 10:40:00,0.0,115,555.96,1440.0


In [ ]:
display(merged_historical)

In [ ]:
# calculate statistics for each storm duration based on full rolling input of ~past~ data
past_mean_pr = 0
past_std_pr = 0
# print(past_std_pr)
past_mean_pr = past_max_each_year.groupby(['duration']).mean()
# print(copy_past_max_each_year)
past_std_pr  = past_max_each_year.groupby(['duration']).std()
euler_mascheroni_constant = 0.57721566490153286060651209008240243104215933593992

# calculate shape factors from statistics for each duration
past_beta = past_std_pr * math.sqrt(6) / math.pi
past_mu = past_mean_pr - past_beta * euler_mascheroni_constant

# calculate statistics for each storm duration based on full rolling input of ~future~ data
future_mean_pr = future_max_each_year.groupby(['duration']).mean()
future_std_pr  = future_max_each_year.groupby(['duration']).std()

# calculate shape factors from statistics for each duration
future_beta = future_std_pr * math.sqrt(6) / math.pi
future_mu = future_mean_pr - future_beta * euler_mascheroni_constant

# print(copy_past_max_each_year.max())
# print(past_mean_pr, past_std_pr, future_mean_pr, future_std_pr)
# print(past_beta, past_mu, future_beta, future_mu)

def project(past_beta, past_mu, future_beta, future_mu, merged_historical):
    
#     # remove zero values from each TS  
#     past_rain_only = past_storms_data[past_storms_data['pr'] != 0]
#     future_rain_only = future_storms_data[future_storms_data['pr'] != 0]
#     future_rain_only['pr'] = future_rain_only['pr'] * 1.2
        
#     # calculate statistics for each storm duration based on full rolling input of ~past~ data
#     past_mean_pr = past_rain_only['pr'].mean()
#     past_std_pr  = past_rain_only['pr'].std()
#     euler_mascheroni_constant = 0.57721566490153286060651209008240243104215933593992

    
#     print(past_mean_pr, past_std_pr)
#     print(np.exp(-1 * np.exp(-1 * (value / merged_historical.loc[count, 'duration']) - float(past_mu.loc[merged_historical['duration'].iloc[count]])))/ float(past_beta.loc[merged_historical['duration'].iloc[count]]))

    for count, value in enumerate(merged_historical['pr']):
# #         print(past_beta.loc[past_max_each_year['duration'].iloc[count]])
# #         print(past_mu.loc[merged_historical['duration'].iloc[count]])
#         print(math.exp(-1 * math.exp(-1 * (value - float(past_mu.loc[merged_historical['duration'].iloc[count]])) / float(past_beta.loc[merged_historical['duration'].iloc[count]]))))
        merged_historical.loc[count, 'CDF'] = math.exp(-1 * math.exp(-1 * (value - float(past_mu.loc[merged_historical['duration'].iloc[count]])) / float(past_beta.loc[merged_historical['duration'].iloc[count]])))
#         merged_historical.loc[count, 'projected_pr_sum'] = float(-1 * float(future_beta.loc[merged_historical['duration'].iloc[count]]) * (np.log(-1 * np.log(merged_historical.loc[count, 'CDF']))) + float(future_mu.loc[merged_historical['duration'].iloc[count]]))
        merged_historical.loc[count, 'projected_pr_sum'] = -1 * float(future_beta.loc[merged_historical['duration'].iloc[count]]) * (math.log(-1 * math.log(float(merged_historical.loc[count, 'CDF'])))) + float(future_mu.loc[merged_historical['duration'].iloc[count]])
#         merged_historical.loc[count, 'delta'] = merged_historical.projected_pr_sum[count] / projected.pr_sum[count]
#         merged_historical.loc[count, 'projected_pr'] = merged_historical.delta * merged_historical.pr

        
#         merged_historical.loc[count, 'delta'] = merged_historical.loc[count, 'projected_pr'] / merged_historical.loc[count, 'pr_sum']
#         merged_historical.loc[count, 'Projected pr'] = merged_historical.loc[count, 'pr'] * merged_historical.loc[count, 'delta']

# merged_historical.loc[count, 'delta'] = math.exp(-1 * math.exp(-1 * value - past_mu.loc[merged_historical['duration'].iloc[count]])) / past_beta.loc[merged_historical['duration'].iloc[count]]
#         merged_historical.loc[count, 'CDF'] = float(math.exp(-1 * math.exp(-1 * (value - past_mu.loc[past_max_each_year['duration'].iloc[count]]) / past_beta.loc[past_max_each_year['duration'].iloc[count]])))
#         merged_historical.loc[count, 'projected_pr'] = float(-1 * future_beta.loc[past_max_each_year['duration'].iloc[count]] * (math.log(-1 * math.log(past_max_each_year.loc[count, 'CDF']))) + future_mu.loc[past_max_each_year['duration'].iloc[count]])
#         print(past_max_each_year.loc[count, 'CDF'])
#         past_max_each_year.loc[count, 'projected_pr'] = future_beta.loc[past_max_each_year['duration'].iloc[count]]
#         print(-1 * future_beta.loc[past_max_each_year['duration'].iloc[count]] * (math.log(-1 * math.log(past_max_each_year.loc[count, 'CDF']))) + future_mu.loc[past_max_each_year['duration'].iloc[count]])
#         print(future_beta.loc[past_max_each_year['duration'].iloc[count]], future_mu.loc[past_max_each_year['duration'].iloc[count]], past_max_each_year.loc[count, 'CDF'])
    
#         past_max_each_year.loc[count, 'delta'] = past_max_each_year.loc[count, 'projected_pr'] / past_max_each_year.loc[count, 'pr_sum']    
    return(merged_historical)
projected = 0
projected = project(past_beta, past_mu, future_beta, future_mu, merged_historical)
projected
# float(-1 * future_beta.loc[past_max_each_year['duration'].iloc[count]] * (math.log(-1 * math.log(past_max_each_year.loc[count, 'CDF']))) + future_mu.loc[past_max_each_year['duration'].iloc[count]]

In [ ]:
# calculate statistics for each storm duration based on full rolling input of ~past~ data
past_mean_pr = 0
past_std_pr = 0
# print(past_std_pr)
past_mean_pr = past_max_each_year.groupby(['duration']).mean()
# print(copy_past_max_each_year)
past_std_pr  = past_max_each_year.groupby(['duration']).std()
euler_mascheroni_constant = 0.57721566490153286060651209008240243104215933593992

# calculate shape factors from statistics for each duration
past_beta = past_std_pr * math.sqrt(6) / math.pi
past_mu = past_mean_pr - past_beta * euler_mascheroni_constant

# calculate statistics for each storm duration based on full rolling input of ~future~ data
future_mean_pr = future_max_each_year.groupby(['duration']).mean()
future_std_pr  = future_max_each_year.groupby(['duration']).std()

# calculate shape factors from statistics for each duration
future_beta = future_std_pr * math.sqrt(6) / math.pi
future_mu = future_mean_pr - future_beta * euler_mascheroni_constant

# print(copy_past_max_each_year.max())
# print(past_mean_pr, past_std_pr, future_mean_pr, future_std_pr)
# print(past_beta, past_mu, future_beta, future_mu)

def project(past_beta, past_mu, future_beta, future_mu, merged_historical):
    
#     # remove zero values from each TS  
#     past_rain_only = past_storms_data[past_storms_data['pr'] != 0]
#     future_rain_only = future_storms_data[future_storms_data['pr'] != 0]
#     future_rain_only['pr'] = future_rain_only['pr'] * 1.2
        
#     # calculate statistics for each storm duration based on full rolling input of ~past~ data
#     past_mean_pr = past_rain_only['pr'].mean()
#     past_std_pr  = past_rain_only['pr'].std()
#     euler_mascheroni_constant = 0.57721566490153286060651209008240243104215933593992

    
#     print(past_mean_pr, past_std_pr)
#     print(np.exp(-1 * np.exp(-1 * (value / merged_historical.loc[count, 'duration']) - float(past_mu.loc[merged_historical['duration'].iloc[count]])))/ float(past_beta.loc[merged_historical['duration'].iloc[count]]))

    for count, value in enumerate(merged_historical.duration):
# #         print(past_beta.loc[past_max_each_year['duration'].iloc[count]])
# #         print(past_mu.loc[merged_historical['duration'].iloc[count]])
#         print(math.exp(-1 * math.exp(-1 * (value - float(past_mu.loc[merged_historical['duration'].iloc[count]])) / float(past_beta.loc[merged_historical['duration'].iloc[count]]))))
        merged_historical.loc[count, 'CDF'] = math.exp(-1 * math.exp(-1 * (value - float(past_mu.loc[merged_historical['duration'].iloc[count]])) / float(past_beta.loc[merged_historical['duration'].iloc[count]])))
#         merged_historical.loc[count, 'projected_pr_sum'] = float(-1 * float(future_beta.loc[merged_historical['duration'].iloc[count]]) * (np.log(-1 * np.log(merged_historical.loc[count, 'CDF']))) + float(future_mu.loc[merged_historical['duration'].iloc[count]]))
        merged_historical.loc[count, 'projected_pr'] = -1 * float(future_beta.loc[merged_historical['duration'].iloc[count]]) * (math.log(-1 * math.log(float(merged_historical.loc[count, 'CDF'])))) + float(future_mu.loc[merged_historical['duration'].iloc[count]])

#         merged_historical.loc[count, 'delta'] = merged_historical.loc[count, 'projected_pr'] / merged_historical.loc[count, 'pr_sum']
#         merged_historical.loc[count, 'Projected pr'] = merged_historical.loc[count, 'pr'] * merged_historical.loc[count, 'delta']

# merged_historical.loc[count, 'delta'] = math.exp(-1 * math.exp(-1 * value - past_mu.loc[merged_historical['duration'].iloc[count]])) / past_beta.loc[merged_historical['duration'].iloc[count]]
#         merged_historical.loc[count, 'CDF'] = float(math.exp(-1 * math.exp(-1 * (value - past_mu.loc[past_max_each_year['duration'].iloc[count]]) / past_beta.loc[past_max_each_year['duration'].iloc[count]])))
#         merged_historical.loc[count, 'projected_pr'] = float(-1 * future_beta.loc[past_max_each_year['duration'].iloc[count]] * (math.log(-1 * math.log(past_max_each_year.loc[count, 'CDF']))) + future_mu.loc[past_max_each_year['duration'].iloc[count]])
#         print(past_max_each_year.loc[count, 'CDF'])
#         past_max_each_year.loc[count, 'projected_pr'] = future_beta.loc[past_max_each_year['duration'].iloc[count]]
#         print(-1 * future_beta.loc[past_max_each_year['duration'].iloc[count]] * (math.log(-1 * math.log(past_max_each_year.loc[count, 'CDF']))) + future_mu.loc[past_max_each_year['duration'].iloc[count]])
#         print(future_beta.loc[past_max_each_year['duration'].iloc[count]], future_mu.loc[past_max_each_year['duration'].iloc[count]], past_max_each_year.loc[count, 'CDF'])
    
#         past_max_each_year.loc[count, 'delta'] = past_max_each_year.loc[count, 'projected_pr'] / past_max_each_year.loc[count, 'pr_sum']    
    return(merged_historical)
projected = 0
projected = project(past_beta, past_mu, future_beta, future_mu, merged_historical)
projected
# float(-1 * future_beta.loc[past_max_each_year['duration'].iloc[count]] * (math.log(-1 * math.log(past_max_each_year.loc[count, 'CDF']))) + future_mu.loc[past_max_each_year['duration'].iloc[count]]

In [ ]:
def project(past_beta, past_mu, future_beta, future_mu, merged_historical):

    past_beta = past_beta.loc[360]
    past_mu = past_mu.loc[360]
    future_beta = future_beta.loc[360]
    future_mu = future_mu.loc[360]
    
    for count, value in enumerate(merged_historical[merged_historical.duration == 360].index):
#         merged_historical.loc[value, 'CDF'] = math.exp(-1 * math.exp(-1 * (merged_historical.pr[value] - past_mu) / past_beta))
#         merged_historical.loc[value, 'projected_pr'] = -1 * future_beta * (math.log(-1 * math.log(merged_historical.loc[value, 'CDF']))) + future_mu
        merged_historical.loc[value, 'projected_pr'] = value #-1 * future_beta * (math.log(-1 * math.log(merged_historical.loc[value, 'CDF']))) + future_mu



#         merged_historical.loc[count, 'CDF'] = math.exp(-1 * math.exp(-1 * (value - past_mu) / past_beta))
#         merged_historical.loc[count, 'projected_pr'] = -1 * future_beta * (math.log(-1 * math.log(merged_historical.loc[count, 'CDF']))) + future_mu

    return(merged_historical)
projected = 0
projected = project(past_beta, past_mu, future_beta, future_mu, merged_historical)
projected
# float(-1 * future_beta.loc[past_max_each_year['duration'].iloc[count]] * (math.log(-1 * math.log(past_max_each_year.loc[count, 'CDF']))) + future_mu.loc[past_max_each_year['duration'].iloc[count]]

In [ ]:
# print(projected.projected_pr_sum.max())
projected.projected_pr_sum = abs(projected.projected_pr_sum)
print([i for i in projected.projected_pr_sum])

# merged_historical['projected_pr_sum']
# merged_historical['delta'] = merged_historical.projected_pr_sum / merged_historical.pr_sum
# merged_historical['projected_pr'] = merged_historical.pr * merged_historical.delta

# merged_historical

In [ ]:
#         merged_historical.loc[count, 'Projected pr'] = merged_historical.loc[count, 'pr'] * merged_historical.loc[count, 'delta']
projected['delta'] = projected.projected_pr_sum / projected.pr_sum
projected['projected_pr'] = projected.delta * projected.pr

# print(projected.pr * projected.delta)
# for count, i in enumerate(projected.pr):
#     projected.iloc[count, 'projected_pr'] = i * projected.delta[count])

# projected['projected_pr'] = projected.pr * projected.delta
# projected.loc['NaN'] = 0
projected

In [ ]:
projected.iloc[640:670] 

In [ ]:
def project_by_intensity(past_storms_data, future_storms_data, historical):
    
    # remove zero values from each TS  
    past_rain_only = past_storms_data[past_storms_data['pr'] != 0]
    future_rain_only = future_storms_data[future_storms_data['pr'] != 0]
#     future_rain_only['pr'] = future_rain_only['pr'] * 1.2
        
    # calculate statistics for each storm duration based on full rolling input of ~past~ data
    past_mean_pr = past_rain_only['pr'].mean()
    past_std_pr  = past_rain_only['pr'].std()
    euler_mascheroni_constant = 0.57721566490153286060651209008240243104215933593992
    
    # calculate shape factors from statistics for each duration
    past_beta = past_std_pr * math.sqrt(6) / math.pi
    past_mu = past_mean_pr - past_beta * euler_mascheroni_constant

#     print(past_mean_pr)
#     print(past_std_pr)
    # print(past_beta)

    # calculate statistics for each storm duration based on full rolling input of ~future~ data
    future_mean_pr = future_rain_only['pr'].mean()
    future_std_pr  = future_rain_only['pr'].std()
#     print(future_mean_pr, future_std_pr)

    # calculate shape factors from statistics for each duration
    future_beta = future_std_pr * math.sqrt(6) / math.pi
    future_mu = future_mean_pr - future_beta * euler_mascheroni_constant
    
#     print(past_beta, past_mu, future_beta, future_mu)
#     print(past_mu.pr)
#     display(historical)
#     print(np.exp(-1 * np.exp(-1 *(14 - past_mu) / past_beta)))
#     print(-1 * future_beta * (math.log(-1 * math.log(.1))) + future_mu)
    for count, value in enumerate(historical.loc[ : ,'pr']):
        historical.loc[count, 'CDF'] = math.exp(-1 * math.exp(-1 * (value - float(past_mu)) / float(past_beta)))
        historical.loc[count, 'Projected pr'] = -1 * future_beta * (math.log(-1 * math.log(historical.loc[count, 'CDF']))) + future_mu
#         historical.loc[count, 'Delta'] = historical.loc[count, 'Projected pr'] / historical.loc[count, 'pr']


    
    return historical
projected_by_intensity = project_by_intensity(past_storms_data, future_storms_data, historical)
display(projected_by_intensity)

In [ ]:
x1 = historical['datetime']
y1 = historical['pr']

x2 = x1
y2 = projected_by_intensity['Projected pr']

x3 = projected.datetime
y3 = projected.projected_pr

plt.rcParams.update({'font.size': 24})

# Create the line plot with different colors
plt.figure(figsize=(18,15))
plt.plot(x2, y2, label='Individual Intensity Projection', color = 'orange')
plt.plot(x1, y1, label='Historical (Observed) Precipitation')
plt.plot(x3, y3, label='Storm-Storm Projection', color = 'green')

# plt.xlim(15641, 15643.5)  # Set x-axis limits from 0 to 20
# plt.ylim(-.25, 7)   # Set y-axis limits from -2 to 20 

# Add labels and title
plt.xlabel("Date", fontsize = 32)
plt.ylabel("Precipitation (mm/hr)", fontsize = 32)
# plt.title("Coarse Time Step", fontsize = 36)

handles, labels = plt.gca().get_legend_handles_labels()
order = [1,0,2]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order])

# Add legend
# plt.legend()

# Show the plot
plt.show()

In [ ]:
x1 = historical['datetime']
y1 = historical['pr']

x2 = x1
y2 = projected_by_intensity['Projected pr']

x3 = projected.datetime
y3 = projected.projected_pr

plt.rcParams.update({'font.size': 24})

# Create the line plot with different colors
plt.figure(figsize=(15,10))
plt.plot(x2, y2, label='Intensity Projection')
plt.plot(x1, y1, label='Historical (Observed) Precipitation')
plt.plot(x3, y3, label='Storm-Storm Projection')

plt.xlim(15641, 15643.5)  # Set x-axis limits from 0 to 20
# plt.ylim(-.25, 7)   # Set y-axis limits from -2 to 20 

# Add labels and title
plt.xlabel("Date", fontsize = 32)
plt.ylabel("Precipitation (mm/hr)", fontsize = 32)
# plt.title("Coarse Time Step", fontsize = 36)

# Add legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
len(projected.index)

In [ ]:
storms_data = read_csv('sample_past_precip.csv')
storms_data = read_netcdf4('CONUS404-19791001-20221001-Dover-Air-Force-Base.nc')
TS = find_timestep(storms_data)
max_each_year = separate_into_durations(storms_data, stormDurations)
past_CDF = compute_CDF(max_each_year, stormDurations)
historical = read_historical_csv('2012_historical.csv')
hist_TS = find_timestep(historical)
discretize_by_storm(historical, dry_time_between_storms)

